In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE208143"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE208143"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE208143.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE208143.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE208143.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE208143_family.soft.gz', 'GSE208143_series_matrix.txt.gz']
Identified SOFT files: ['GSE208143_family.soft.gz']
Identified matrix files: ['GSE208143_series_matrix.txt.gz']

Background Information:
!Series_title	"mRNA expression profile from retinoblastoma tumors and pediatric controls"
!Series_summary	"To discover differentially expressed mRNA's in Rb tumors compared to pediatric retina"
!Series_overall_design	"Nine enucleated human retinoblastoma tumors and two pediatric retina controls used for the study. Total RNA was isolated from 9 Rb tumors and 2 control pediatric retina samples using Agilent Absolutely RNA miRNA kit. Twenty-five nanograms of RNA from Rb tumors and control pediatric retina samples were labeled with Cy3 dye using an Agilent Low Input Quick Amp Labeling Kit"

Sample Characteristics Dictionary:
{0: ['tissue: Tumor', 'tissue: Pediatric Retina'], 1: ['gender: Male', 'gender: Female']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset is looking at mRNA expression profiles in retinoblastoma tumors vs controls
# The dataset mentions "Total RNA was isolated", "mRNA expression profile"
# indicating that it contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Retinoblastoma):
# From the sample characteristics, key 0 has 'tissue: Tumor' and 'tissue: Pediatric Retina'
# which indicates Retinoblastoma status (Tumor vs Control)
trait_row = 0

# For gender:
# From the sample characteristics, key 1 has 'gender: Male' and 'gender: Female'
gender_row = 1

# For age:
# There is no age information in the sample characteristics dictionary
age_row = None

# 2.2 Data Type Conversion

# Convert tissue type to binary (Tumor = 1, Control = 0)
def convert_trait(value):
    if isinstance(value, str):
        value = value.lower()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'tumor' in value:
            return 1
        elif 'retina' in value or 'control' in value:
            return 0
    return None

# Convert gender to binary (Male = 1, Female = 0)
def convert_gender(value):
    if isinstance(value, str):
        value = value.lower()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'male' in value:
            return 1
        elif 'female' in value:
            return 0
    return None

# Age conversion function (not used in this dataset)
def convert_age(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Import necessary function
        from tools.preprocess import get_feature_data
        
        # The sample characteristics dictionary from the previous output
        sample_characteristics_dict = {0: ['tissue: Tumor', 'tissue: Pediatric Retina'], 
                                      1: ['gender: Male', 'gender: Female']}
        
        # Create DataFrame from the sample characteristics
        sample_ids = [f"Sample_{i+1}" for i in range(len(sample_characteristics_dict[0]))]
        clinical_data = pd.DataFrame(index=sample_ids)
        
        # Add each feature as a column
        for row_idx, values in sample_characteristics_dict.items():
            feature_name = f"Feature_{row_idx}"
            clinical_data[feature_name] = values
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:")
        print(preview)
        
        # Save the clinical features to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=False)
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        print("Unable to extract clinical features. Skipping this step.")


Preview of clinical features:
{'Feature_0': [1.0, nan], 'Feature_1': [nan, 1.0]}


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['A_19_P00315452', 'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492',
       'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518',
       'A_19_P00315519', 'A_19_P00315524', 'A_19_P00315528', 'A_19_P00315529',
       'A_19_P00315538', 'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315550',
       'A_19_P00315551', 'A_19_P00315554', 'A_19_P00315581', 'A_19_P00315583'],
      dtype='object', name='ID')

Gene expression data shape: (50521, 33)


### Step 4: Gene Identifier Review

In [5]:
# Examine the identifiers in the gene expression data
# The identifiers starting with "A_19_P" appear to be Agilent microarray probe IDs
# and not standard human gene symbols

# These are probe identifiers from an Agilent microarray platform
# They need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From the gene annotation preview, we can see:
# - 'ID' contains the probe identifiers like A_23_P117082 which match gene_data's index
# - 'GENE_SYMBOL' contains the human gene symbols like HEBP1

# 2. Get a gene mapping dataframe using the library function which ensures proper column naming
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few mapping entries:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# We'll use the library function to handle the many-to-many mapping scenario
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nAfter mapping, gene expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Let's also normalize gene symbols to ensure consistency across datasets
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nAfter normalization, gene expression data shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (46204, 2)
First few mapping entries:
              ID          Gene
2   A_23_P117082         HEBP1
3  A_33_P3246448         KCNE4
4  A_33_P3318220        BPIFA3
5  A_33_P3236322  LOC100129869
6  A_33_P3319925          IRG1

After mapping, gene expression data shape: (20330, 33)
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT',
       'A4GNT', 'AA06'],
      dtype='object', name='Gene')



After normalization, gene expression data shape: (19825, 33)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT',
       'AA06', 'AAA1'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE208143.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Re-extract and normalize gene data
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Re-extract gene expression data
gene_data = get_genetic_data(matrix_file_path)
print(f"Gene expression data shape: {gene_data.shape}")

# Re-extract gene annotation
gene_annotation = get_gene_annotation(soft_file_path)

# Get gene mapping and apply it
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Get clinical data
background_info, clinical_data = get_background_and_clinical_data(matrix_file_path)

# Define conversion functions
def convert_trait(value):
    if isinstance(value, str):
        value = value.lower()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'tumor' in value:
            return 1
        elif 'retina' in value or 'control' in value:
            return 0
    return None

def convert_gender(value):
    if isinstance(value, str):
        value = value.lower()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'male' in value:
            return 1
        elif 'female' in value:
            return 0
    return None

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,  # As identified in Step 2
    convert_trait=convert_trait,
    gender_row=1,
    convert_gender=convert_gender
)

print(f"Clinical features shape: {clinical_features.shape}")
print("Clinical features preview:")
print(preview_df(clinical_features))

# Save the clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values, linked data shape: {linked_data.shape}")

# 4. Determine if trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
note = "Dataset contains gene expression data from retinoblastoma tumors and pediatric retina controls."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,
    is_trait_available=True,  # We have trait data (tumor vs control)
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 6. Save linked data if usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Gene expression data shape: (50521, 33)


Normalized gene data shape: (19825, 33)


Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE208143.csv
Clinical features shape: (2, 33)
Clinical features preview:
{'GSM6338046': [1.0, 1.0], 'GSM6338047': [1.0, 1.0], 'GSM6338048': [1.0, 1.0], 'GSM6338049': [1.0, 1.0], 'GSM6338050': [1.0, 1.0], 'GSM6338051': [1.0, 1.0], 'GSM6338052': [1.0, 1.0], 'GSM6338053': [1.0, 1.0], 'GSM6338054': [1.0, 1.0], 'GSM6338055': [1.0, 1.0], 'GSM6338056': [1.0, 1.0], 'GSM6338057': [1.0, 1.0], 'GSM6338058': [1.0, 1.0], 'GSM6338059': [1.0, 1.0], 'GSM6338060': [1.0, 1.0], 'GSM6338061': [1.0, 1.0], 'GSM6338062': [1.0, 1.0], 'GSM6338063': [1.0, 1.0], 'GSM6338064': [1.0, 1.0], 'GSM6338065': [1.0, 1.0], 'GSM6338066': [1.0, 1.0], 'GSM6338067': [1.0, 1.0], 'GSM6338068': [1.0, 1.0], 'GSM6338069': [1.0, 1.0], 'GSM6338070': [1.0, 1.0], 'GSM6338071': [1.0, 1.0], 'GSM6338072': [1.0, 1.0], 'GSM6338073': [0.0, 1.0], 'GSM6338074': [0.0, 1.0], 'GSM6338075': [0.0, 1.0], 'GSM6338076': [0.0, 1.0], 'GSM6338077': [0.0, 1.0], 'GSM6338078': 

After handling missing values, linked data shape: (33, 19827)
For the feature 'Retinoblastoma', the least common label is '0.0' with 6 occurrences. This represents 18.18% of the dataset.
The distribution of the feature 'Retinoblastoma' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 33 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



A new JSON file was created at: ../../output/preprocess/Retinoblastoma/cohort_info.json
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Retinoblastoma/GSE208143.csv
